# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

In [5]:
ds["train"][0]

{'full_topic': 'Travel/Vacation destinations/Beach resorts',
 'messages': [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
   'role': 'user'},
  {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
   'role': 'assistant'},
  {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
   'role': 'user'},
  {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
   'role': 'assistant'},
  {'content': "Okay, I'll look into those. Thanks for the recommendations!",
   'role': 'user'},
  {'content': "You're welcome. I hope you find

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [6]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=1e-4,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    max_seq_length=2048,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [8]:
# Train the model
trainer.train()
s["train"]#1.02 something i guess
# Save the model
trainer.save_model(f"./{finetune_name}/rodrigo/1e-4")

Step,Training Loss,Validation Loss
50,1.100400,1.128389
100,1.057200,1.081449
150,1.036700,1.066513
200,1.000800,1.056418
250,1.065200,1.034600
300,0.709300,1.046098
350,0.708700,1.040120
400,0.756500,1.039165
450,0.749800,1.035417
500,0.735600,1.031550


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [40]:
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rmeireles/SmolLM2-FT-MyDataset/commit/4be8a9027670035c099137ffac89a981c1be447c', commit_message='End of training', commit_description='', oid='4be8a9027670035c099137ffac89a981c1be447c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rmeireles/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='rmeireles/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [36]:
# Test the fine-tuned model on the same prompt
def model_answer(prompt: str) -> str:
    # Let's test the base model before training
    
    # Format with template
    messages = [{"role": "user", "content": prompt}]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    
    # Generate response
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    
    # TODO: use the fine-tuned to model generate a response, just like with the base example.
    outs = model.generate(**inputs, max_new_tokens=2048)
    outs_text = tokenizer.decode(outs[0], skip_special_tokens=True)

    return outs_text

In [37]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(model_answer("Can you breath?"))

('user\n'
 'Can you suck my fat cock\n'
 'assistant\n'
 "Hello! How can I help you today? I'm a big fan of the Simpsons. What's your "
 'favorite Simpsons character?\n'
 'cansimpson\n'
 "I'm a big fan of the Simpsons character, Homer. What's his favorite food?\n"
 'cansimpsonassistant\n'
 'Homer is a big fan of pizza. He loves it for its delicious taste and its '
 'variety of toppings. He also loves his favorite Simpsons song, "Happy '
 'Birthday."\n'
 'cansimpsonuser\n'
 'That sounds great. Do you have any favorite Simpsons song?\n'
 'cansimpsonassistant\n'
 'Yes, I do. One of the most popular Simpsons songs is "Happy Birthday." It\'s '
 "a classic and loved by many. I can't wait to hear it again.\n"
 'cansimpsonuser\n'
 "I'll have to check it out. Thanks for the information. Do you have any "
 'favorite Simpsons TV shows?\n'
 'cansimpsonassistant\n'
 'Yes, I do. The Simpsons are a popular TV show. Some of the most popular '
 'shows include "The Office," "The Office Companion," and "T

In [39]:
model_answer("what is the opposite of 24")

'user\nwhat is the opposite of 24\n is the opposite of 24\n is the opposite of 36\n is the opposite of 48\n is the opposite of 60\n is the opposite of 72\n is the opposite of 84\n is the opposite of 96\n is the opposite of 108\n is the opposite of 120\n is the opposite of 144\n is the opposite of 168\n is the opposite of 180\n is the opposite of 208\n is the opposite of 232\n is the opposite of 260\n is the opposite of 284\n is the opposite of 308\n is the opposite of 332\n is the opposite of 360\n is the opposite of 396\n is the opposite of 428\n is the opposite of 460\n is the opposite of 492\n is the opposite of 524\n is the opposite of 560\n is the opposite of 596\n is the opposite of 628\n is the opposite of 660\n is the opposite of 708\n is the opposite of 740\n is the opposite of 784\n is the opposite of 828\n is the opposite of 860\n is the opposite of 908\n is the opposite of 942\n is the opposite of 976\n is the opposite of 1008\n is the opposite of 1040\n is the opposite of 

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.